<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/Classical%20Transportation%20Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classical Transportation Problem

## Problem Description

The classic(al) transportation problem concerns minimizing the cost of transporting a product from sources/supplies to destinations/demands. The three inputs of the model are total units produced at each source, total units needed at each destination, and the cost to transport one unit from each source to each destination. And the objective is to minimize the total cost of transporting all units produced at sources to meet the demands at destinations. (Source: https://gistbok.ucgis.org/bok-topics/classic-transportation-problem)


**Example:** <br>
Container need to be transported from sources (Kiel, Magdeburg, Dortmund) to destinations (Hamburg, Berlin, Cologne, Munich). The available containers (supply) in Kiel, Magdeburg, and Dortmund are 45, 120, and 9, respectively. The demands in Hamburg, Berlin, Cologne, Munich are 80, 78, 47, and 55, respectively The transportation costs in tsd. Euro per container are shown in the following table:

<br>
$
\begin{array}{lrrrrrrr}
    \hline
    \text{} & \text{Hamburg} & \text{Berlin} & \text{Cologne} & \text{Munich} \\ \hline
     \text{Kiel} & 20 & 40 & 50 & 120 \\
     \text{Magdeburg} & 30 & 10 & 40 & 60 \\
     \text{Dortmund} & 45 & 40 & 10 & 50 \\
     \hline
\end{array}
$
<br><br>

How to minimize the total cost of transporting all units produced at sources to meet the demands at destinations?



## Mathematical Program

**Notation:**
\begin{array}{ll}
A & \text{Set an Angebotsknoten}\\
N & \text{Set an Nachfrageknoten}\\
c_{ij}  & \text{Transportkosten zwischen Angebotsknoten $i \in A$ und Nachfrageknoten $j \in N$}\\
\end{array}
<br>

**Decision variables:**
\begin{array}{ll}
x_{ij} & \text{Transportmenge zwischen Angebotsknoten $i \in A$ und Nachfrageknoten $j \in N$}\\
\end{array}
<br>

**Objective:** <br>
$
\begin{align}
  \begin{array}{llll}
    & \min \sum\limits_{i \in A} \sum\limits_{j \in N} c_{ij} \cdot x_{ij} & &~~~  (1) \\
  \end{array}
\end{align}
$
<br>

**Constraints:**<br>
$
\begin{align}
  \begin{array}{llll}
    & a_i = \sum\limits_{j \in N} x_{ij} &,~ \forall~ i \in A &~~~ (2) \\
    & b_j = \sum\limits_{i \in A} x_{ij} &,~ \forall~ j \in N &~~~ (3) \\
    & x_{ij} \ge 0 &,~ \forall~ i \in A, j \in N &~~~ (4) \\
  \end{array}
\end{align}
$

## Implementation

### Data

In [ ]:
# Angebotsknoten
A = ['Kiel', 'Magdeburg', 'Dortmund']

# Angebotsmenge
a = {'Kiel': 45, 'Magdeburg': 120, 'Dortmund': 95}

# Nachfrageknoten
N = ['Hamburg', 'Berlin', 'Cologne', 'Munich']

# Nachfragemenge
b = {'Hamburg': 80, 'Berlin': 78, 'Cologne': 47, 'Munich': 55}

# Kosten
c = {
    'Kiel': {'Hamburg': 20, 'Berlin': 40, 'Cologne': 50, 'Munich': 120},
    'Magdeburg': {'Hamburg': 30, 'Berlin': 10, 'Cologne': 40, 'Munich': 60},
    'Dortmund': {'Hamburg': 45, 'Berlin': 40, 'Cologne': 10, 'Munich': 50},
}

### PuLP model

In [ ]:
!pip install pulp
import pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 54.6 MB/s eta 0:00:00


In [ ]:
# Model
model = pulp.LpProblem(name='Transportproblem',
                       sense=pulp.constants.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts(name='x', indices=(A, N), lowBound=0, cat='Integer')

# (1) Objective
model += pulp.lpSum(c[i][j] * x[i][j] for i in A for j in N), '(1)'

# (2)
for i in A:
  model += a[i] == pulp.lpSum(x[i][j] for j in N), '(2)_' + str(i)

# (2)
for j in N:
  model += b[j] == pulp.lpSum(x[i][j] for i in A), '(3)_' + str(j)

# print(model)

### Solution

In [ ]:
# solve problem
model.solve()

# get status
print("Status:", pulp.LpStatus[model.status])

# get objective value
print('Objective value:', round(pulp.value(model.objective), 2))

# get value of decision variable u (position in tour of the nodes that are part of the tour)
for i in A:
  for j in N:
    if x[i][j].varValue > 0:
      print('From ' + str(i) + ' to ' + str(j) + ': ' + str(x[i][j].varValue) + ' ME')


Status: Optimal
Objective value: 219900.0
Von Sunnyvale nach Falls: 45.0 ME
Von Dublin nach Teaneck: 78.0 ME
Von Dublin nach Chicago: 42.0 ME
Von Bankok nach Amarillo: 80.0 ME
Von Bankok nach Chicago: 5.0 ME
Von Bankok nach Falls: 10.0 ME
